In [69]:
import pandas as pd
import random

In [70]:
csv = pd.read_csv('input/delitos_2016.csv')
for i in range(2017, 2021):
    csv = pd.concat([csv, pd.read_csv(f'input/delitos_{i}.csv')])

/var/folders/zk/0yzb9y8s6z71smfld5fjvdjh0000gn/T/ipykernel_14488/527257168.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.concat([csv, pd.read_csv(f'input/delitos_{i}.csv')])


In [71]:
# convert DD/MM/AAAA to DD-MM-AAAA
csv['fecha'] = csv['fecha'].apply(lambda x: x.replace('/', '-'))
# change lat/long to latitud/longitud
csv = csv.rename(columns={'lat': 'latitud', 'long': 'longitud'})
# if uso_armas is not present or null, set to "Sin Datos"
csv['uso_armas'] = csv['uso_armas'].fillna('Sin Datos')
# if subtipo is not present or null, set to "Sin Datos"
csv['subtipo'] = csv['subtipo'].fillna('Robo')
# remove id and id_mapa
csv = csv.drop(columns=['id', 'id_mapa', 'cantidad_registrada'])
# set anio, mes, dia with the fecha column
csv['anio'] = csv['fecha'].apply(lambda x: int(x.split('-')[2]))
csv['mes'] = csv['fecha'].apply(lambda x: int(x.split('-')[1]))
csv['nro_dia'] = csv['fecha'].apply(lambda x: int(x.split('-')[0]))
# set comuna to int
# csv['comuna'] = csv['comuna'].apply(lambda x: int(x))

In [72]:
csv.head()

,fecha,franja,tipo,subtipo,comuna,barrio,latitud,longitud,mes,dia,anio,uso_armas,victimas,nro_dia
0,01-01-2016,13,Robo (con violencia),Robo,9.0,Parque Avellaneda,-34.64873,-58.468536,1,Viernes,2016,Sin Datos,NaN,1
1,01-01-2016,22,Robo (con violencia),Robo,14.0,Palermo,-34.576925,-58.4074,1,Viernes,2016,Sin Datos,NaN,1
2,01-01-2016,1,Lesiones,Siniestro Vial,4.0,Parque Patricios,-34.645197,-58.401099,1,Viernes,2016,Sin Datos,NaN,1
3,01-01-2016,19,Hurto (sin violencia),Robo,9.0,Parque Avellaneda,-34.65576,-58.478681,1,Viernes,2016,Sin Datos,NaN,1
4,01-01-2016,4,Robo (con violencia),Robo Automotor,6.0,Caballito,-34.624128,-58.436826,1,Viernes,2016,Sin Datos,NaN,1


In [73]:
# TABLA TIPOS
tipos_csv = csv[['tipo', 'subtipo']]
tipos_csv = tipos_csv.drop_duplicates(ignore_index=True)
tipos_csv = tipos_csv.fillna('Robo')
tipos_csv['id'] = tipos_csv.index + 1

In [74]:
# TABLA TIEMPOS
tiempos_csv = csv[['anio', 'mes', 'dia', 'fecha', 'franja']]
tiempos_csv['nro_dia'] = tiempos_csv['fecha'].apply(lambda x: int(x.split('-')[0]))
tiempos_csv = tiempos_csv.drop_duplicates(ignore_index=True)
tiempos_csv['id'] = tiempos_csv.index + 1
tiempos_csv['dia'] = tiempos_csv['dia'].fillna('Sin Datos')
tiempos_csv

/var/folders/zk/0yzb9y8s6z71smfld5fjvdjh0000gn/T/ipykernel_14488/3006930889.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiempos_csv['nro_dia'] = tiempos_csv['fecha'].apply(lambda x: int(x.split('-')[0]))


,anio,mes,dia,fecha,franja,nro_dia,id
0,2016,1,Viernes,01-01-2016,13,1,1
1,2016,1,Viernes,01-01-2016,22,1,2
2,2016,1,Viernes,01-01-2016,1,1,3
3,2016,1,Viernes,01-01-2016,19,1,4
4,2016,1,Viernes,01-01-2016,4,1,5
...,...,...,...,...,...,...,...
51405,20,12,Jueves,31-12-20,20,31,51406
51406,20,12,Jueves,31-12-20,5,31,51407
51407,20,12,Jueves,31-12-20,12,31,51408
51408,20,12,Jueves,31-12-20,6,31,51409


In [75]:
# TABLA LUGARES
lugares_csv = csv[['barrio', 'comuna', 'latitud', 'longitud']]
# remove the ones that don't have latitud or longitud
lugares_csv = lugares_csv.dropna()
# convert latitud and longitud to numeric if they are not
lugares_csv['latitud'] = pd.to_numeric(lugares_csv['latitud'], errors='coerce')
lugares_csv['longitud'] = pd.to_numeric(lugares_csv['longitud'], errors='coerce')
lugares_csv = lugares_csv.groupby(['barrio', 'comuna']).max()
# if it is smaller than -75 it means there is an error, multiply it by 0.001
lugares_csv['latitud'] = lugares_csv['latitud'].apply(lambda x: x * 0.001 if x < -75 else x)
lugares_csv['longitud'] = lugares_csv['longitud'].apply(lambda x: x * 0.001 if x < -75 else x)
lugares_csv = lugares_csv.reset_index()
lugares_csv['id'] = lugares_csv.index + 1
lugares_csv

,barrio,comuna,latitud,longitud,id
0,Agronomía,15.0,-34.584930,-58.476035,1
1,Almagro,5.0,-34.597733,-58.411991,2
2,Balvanera,3.0,-34.598042,-58.391662,3
3,Barracas,4.0,-34.626654,-58.367685,4
4,Belgrano,13.0,-34.534385,-58.427144,5
5,Boca,4.0,-34.617890,-58.342665,6
6,Boedo,5.0,-34.620738,-58.410514,7
7,Caballito,6.0,-34.602774,-58.426758,8
8,Chacarita,15.0,-34.578295,-58.439178,9
9,Coghlan,12.0,-34.553535,-58.466512,10


In [76]:
# TABLA TIPO DE ARMA
armas_csv = csv[['uso_armas']]
armas_csv = armas_csv.drop_duplicates(ignore_index=True)
armas_csv['id'] = armas_csv.index + 1
armas_csv

,uso_armas,id
0,Sin Datos,1
1,Arma cortante,2
2,Arma de fuego,3


In [77]:
# TABLA DELITOS
delitos_csv = csv
delitos_csv

,fecha,franja,tipo,subtipo,comuna,barrio,latitud,longitud,mes,dia,anio,uso_armas,victimas,nro_dia
0,01-01-2016,13,Robo (con violencia),Robo,9.0,Parque Avellaneda,-34.64873,-58.468536,1,Viernes,2016,Sin Datos,NaN,1
1,01-01-2016,22,Robo (con violencia),Robo,14.0,Palermo,-34.576925,-58.4074,1,Viernes,2016,Sin Datos,NaN,1
2,01-01-2016,1,Lesiones,Siniestro Vial,4.0,Parque Patricios,-34.645197,-58.401099,1,Viernes,2016,Sin Datos,NaN,1
3,01-01-2016,19,Hurto (sin violencia),Robo,9.0,Parque Avellaneda,-34.65576,-58.478681,1,Viernes,2016,Sin Datos,NaN,1
4,01-01-2016,4,Robo (con violencia),Robo Automotor,6.0,Caballito,-34.624128,-58.436826,1,Viernes,2016,Sin Datos,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66720,31-12-20,4,Robo (con violencia),Robo,4.0,Barracas,-34.635561,-58.369959,12,Jueves,20,Sin Datos,NaN,31
66721,31-12-20,16,Robo (con violencia),Robo,1.0,San Telmo,-34.619242,-58.366333,12,Jueves,20,Sin Datos,NaN,31
66722,31-12-20,13,Robo (con violencia),Robo,4.0,Boca,-34.636337,-58.361306,12,Jueves,20,Arma de fuego,NaN,31
66723,31-12-20,4,Robo (con violencia),Robo,4.0,Boca,-34.627914,-58.360876,12,Jueves,20,Sin Datos,NaN,31


In [78]:
# replace uso_armas for arma_id
delitos_csv = delitos_csv.merge(armas_csv, left_on=['uso_armas'], right_on=['uso_armas'])
delitos_csv = delitos_csv.rename(columns={'id': 'arma_id'})
delitos_csv

,fecha,franja,tipo,subtipo,comuna,barrio,latitud,longitud,mes,dia,anio,uso_armas,victimas,nro_dia,arma_id
0,01-01-2016,13,Robo (con violencia),Robo,9.0,Parque Avellaneda,-34.64873,-58.468536,1,Viernes,2016,Sin Datos,NaN,1,1
1,01-01-2016,22,Robo (con violencia),Robo,14.0,Palermo,-34.576925,-58.4074,1,Viernes,2016,Sin Datos,NaN,1,1
2,01-01-2016,1,Lesiones,Siniestro Vial,4.0,Parque Patricios,-34.645197,-58.401099,1,Viernes,2016,Sin Datos,NaN,1,1
3,01-01-2016,19,Hurto (sin violencia),Robo,9.0,Parque Avellaneda,-34.65576,-58.478681,1,Viernes,2016,Sin Datos,NaN,1,1
4,01-01-2016,4,Robo (con violencia),Robo Automotor,6.0,Caballito,-34.624128,-58.436826,1,Viernes,2016,Sin Datos,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560084,31-12-20,11,Robo (con violencia),Robo,4.0,Nueva Pompeya,-34.648257,-58.403787,12,Jueves,20,Arma de fuego,NaN,31,3
560085,31-12-20,12,Robo (con violencia),Robo,2.0,Recoleta,-34.577698,-58.387302,12,Jueves,20,Arma de fuego,NaN,31,3
560086,31-12-20,17,Robo (con violencia),Robo,1.0,Retiro,-34.598079,-58.384101,12,Jueves,20,Arma de fuego,NaN,31,3
560087,31-12-20,18,Robo (con violencia),Robo,4.0,Barracas,-34.642953,-58.379417,12,Jueves,20,Arma de fuego,NaN,31,3


In [79]:
# replace barrio for lugar_id
delitos_csv = delitos_csv.merge(lugares_csv, left_on=['barrio'], right_on=['barrio'])
delitos_csv = delitos_csv.rename(columns={'id': 'lugar_id'})
delitos_csv = delitos_csv.drop(columns=['barrio'])

In [80]:
# replace tipo for tipo_id
delitos_csv = delitos_csv.merge(tipos_csv, left_on=['tipo', 'subtipo'], right_on=['tipo', 'subtipo'])
delitos_csv = delitos_csv.rename(columns={'id': 'tipo_id'})
delitos_csv = delitos_csv.drop(columns=['tipo', 'subtipo'])

In [81]:
# # replace fecha for tiempo_id
delitos_csv = delitos_csv.merge(tiempos_csv, left_on=['fecha', 'franja'], right_on=['fecha', 'franja'])
delitos_csv = delitos_csv.rename(columns={'id': 'tiempo_id'})
delitos_csv = delitos_csv.drop(columns=['fecha'])

In [82]:
# create a victimas column with random values
delitos_csv['victimas'] = delitos_csv.index
delitos_csv['costo'] = delitos_csv.index
delitos_csv['victimas'] = delitos_csv['victimas'].apply(lambda x: random.randint(0, 5))
delitos_csv['costo'] = delitos_csv['costo'].apply(lambda x: random.randint(0, 100000))

In [83]:
delitos_csv = delitos_csv[['lugar_id', 'tiempo_id', 'arma_id', 'tipo_id', 'victimas', 'costo']]
delitos_csv

,lugar_id,tiempo_id,arma_id,tipo_id,victimas,costo
0,23,1,1,1,3,9640
1,6,1,1,1,2,77811
2,3,1,1,3,3,2822
3,23,20,1,1,3,3184
4,19,20,1,1,3,45452
...,...,...,...,...,...,...
551392,41,45467,3,10,3,13141
551393,24,44751,3,10,1,78003
551394,24,45479,3,10,5,453
551395,31,45962,3,10,2,35910


In [84]:
# save csvs to files
tiempos_csv.to_csv('output/tiempos.csv', index=False)
lugares_csv.to_csv('output/lugares.csv', index=False)
armas_csv.to_csv('output/armas.csv', index=False)
delitos_csv.to_csv('output/delitos.csv', index=False)
tipos_csv.to_csv('output/tipos.csv', index=False)